In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf
print(tf.test.gpu_device_name())

/device:GPU:0


In [3]:
import os
import numpy as np
np.random.seed(1142)

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time


import keras.backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, SGD
from keras.models import Model, Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.models import load_model
from keras.applications.inception_v3 import InceptionV3
from sklearn.metrics import accuracy_score, precision_score, f1_score, confusion_matrix

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [4]:
output_classes = 2
batch_size = 64
img_height, img_width = 256, 256
input_shape = (img_height, img_width, 3)
epochs = 15

In [5]:
# if K.image_data_format() == 'channels_first':
#     input_shape = (3, img_width, img_height)
# else:
#     input_shape = (img_width, img_height, 3)

In [6]:
train_dir = 'data/public/train/'
validation_dir = 'data/public/validation'
test_dir = 'data/public/test'

In [7]:
for root,dirs,files in os.walk(train_dir):
    print (root, len(files))

print("*"*30)
for root,dirs,files in os.walk(validation_dir):
    print (root, len(files))
    
print("*"*30)
for root,dirs,files in os.walk(test_dir):
    print (root, len(files))

data/public/train/ 0
data/public/train/malignant 2555
data/public/train/benign 504
******************************
data/public/validation 0
data/public/validation/malignant 341
data/public/validation/benign 66
******************************
data/public/test 0
data/public/test/malignant 387
data/public/test/benign 72


In [8]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = True,
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = True,
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 3059 images belonging to 2 classes.
Found 407 images belonging to 2 classes.
Found 459 images belonging to 2 classes.


In [9]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)
num_classes = len(train_generator.class_indices)

In [10]:
def InceptionV3_model():
    base_model = InceptionV3(input_shape=input_shape, weights='imagenet', include_top=False)
    x = base_model.output
    x = Dense(1024, activation="relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation=tf.nn.softmax)(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    
    return model

model = InceptionV3_model()

In [11]:
space = {
         'lr': hp.choice('lr',[0.001, 0.0001, 0.00001,0.000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
         'beta_1':hp.choice('beta_1',[0.3,0.4,0.5,0.6,0.7,0.8]),
         'beta_2':hp.choice('beta_2',[0.99,0.995,0.7,0.8,0.9,0.999]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
         'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#           'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
#           'epsilon': hp.choice('epsilon',[None,1e-08,1e-06,1e-04,1e-8,1e-9,1e-10,1e-11]),
        }

In [12]:
from keras.backend.tensorflow_backend import get_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import set_session

def reset_keras_tf_session():
    """
    this function clears the gpu memory and set the 
    tf session to not use the whole gpu
    """
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    set_session(tf.Session(config=config))

In [13]:
def f_nn(params):   
    print ('Parameters testing: ', params)


    adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0, amsgrad=params['amsgrad'])
#     sgd_opt=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop_opt=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=params['epsilon'], decay=0.0)

    model.compile(optimizer = adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit_generator(
      train_generator,
      steps_per_epoch = nb_train_samples // batch_size,
      epochs = 15,
      validation_data = validation_generator,
      validation_steps = nb_validation_samples // batch_size,
      verbose = 1)
    
    score = model.evaluate_generator(validation_generator, steps=50)

    print ('Validation Score: ', score[0])
    print ('Validation Accuracy: ',score[1])
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    predicts = model.predict_generator(test_generator, steps=test_generator.samples/test_generator.batch_size, verbose=1)
    predict_class = np.argmax(predicts, axis=1)
    errors = np.where(predict_class != truth)[0]
    print("No of errors = {}/{}".format(len(errors),test_generator.samples))
    
    th = 0.3
    acc = accuracy_score(truth,predict_class > th)
    print('Accuracy:  {:.4f}'.format(acc*100))
    
    print("*_*" * 50)

    sys.stdout.flush() 
#     reset_keras_tf_session()
    
    return {'loss': score[0], 'status': STATUS_OK, 'model': model}

In [14]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=300, trials=trials)
print(best)

Parameters testing:  {'amsgrad': False, 'beta_1': 0.8, 'beta_2': 0.8, 'lr': 0.0001}
Epoch 1/15
47/47 [==============================] - 33s 703ms/step - loss: 0.4648 - acc: 0.8314 - val_loss: 0.4657 - val_acc: 0.8385
Epoch 2/15
47/47 [==============================] - 26s 546ms/step - loss: 0.2806 - acc: 0.8816 - val_loss: 1.1828 - val_acc: 0.8397
Epoch 3/15
47/47 [==============================] - 26s 553ms/step - loss: 0.0546 - acc: 0.9823 - val_loss: 1.1394 - val_acc: 0.7755
Epoch 4/15
47/47 [==============================] - 26s 553ms/step - loss: 0.0403 - acc: 0.9854 - val_loss: 1.0053 - val_acc: 0.8338
Epoch 5/15
47/47 [==============================] - 26s 560ms/step - loss: 0.0455 - acc: 0.9840 - val_loss: 0.9434 - val_acc: 0.7697
Epoch 6/15
47/47 [==============================] - 26s 548ms/step - loss: 0.0362 - acc: 0.9900 - val_loss: 3.0186 - val_acc: 0.3819
Epoch 7/15
47/47 [==============================] - 26s 549ms/step - loss: 0.0168 - acc: 0.9930 - val_loss: 1.8004 - v

47/47 [==============================] - 26s 550ms/step - loss: 0.1139 - acc: 0.9593 - val_loss: 1.7250 - val_acc: 0.8426
Epoch 8/15
47/47 [==============================] - 26s 554ms/step - loss: 0.0609 - acc: 0.9764 - val_loss: 1.2231 - val_acc: 0.8125
Epoch 9/15
47/47 [==============================] - 26s 556ms/step - loss: 0.0427 - acc: 0.9857 - val_loss: 0.8220 - val_acc: 0.7784
Epoch 10/15
47/47 [==============================] - 26s 551ms/step - loss: 0.0633 - acc: 0.9791 - val_loss: 1.2572 - val_acc: 0.8251
Epoch 11/15
47/47 [==============================] - 25s 541ms/step - loss: 0.0600 - acc: 0.9823 - val_loss: 0.5606 - val_acc: 0.7901
Epoch 12/15
47/47 [==============================] - 26s 556ms/step - loss: 0.0801 - acc: 0.9776 - val_loss: 1.7211 - val_acc: 0.8601
Epoch 13/15
47/47 [==============================] - 26s 556ms/step - loss: 0.0341 - acc: 0.9857 - val_loss: 1.0148 - val_acc: 0.8484
Epoch 14/15
47/47 [==============================] - 26s 550ms/step - loss: 

Epoch 14/15
47/47 [==============================] - 26s 559ms/step - loss: 1.2708e-07 - acc: 1.0000 - val_loss: 2.1944 - val_acc: 0.8280
Epoch 15/15
47/47 [==============================] - 27s 572ms/step - loss: 2.5038e-07 - acc: 1.0000 - val_loss: 2.2817 - val_acc: 0.8255
Validation Score:  2.2883769546408774
Validation Accuracy:  0.8245794708033914
8/7 [=================================] - 4s 530ms/step
No of errors = 6/459
Accuracy:  98.6928
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'amsgrad': False, 'beta_1': 0.7, 'beta_2': 0.99, 'lr': 1e-06}
Epoch 1/15
47/47 [==============================] - 35s 742ms/step - loss: 3.3565e-06 - acc: 1.0000 - val_loss: 2.2437 - val_acc: 0.8281
Epoch 2/15
47/47 [==============================] - 26s 547ms/step - loss: 1.9906e-06 - acc: 1.0000 - val_loss: 2.5087 - val_acc: 0.8134
Epoch 3/15
47/47 [======================

47/47 [==============================] - 26s 555ms/step - loss: 9.8269e-06 - acc: 1.0000 - val_loss: 1.1335 - val_acc: 0.8513
Epoch 3/15
47/47 [==============================] - 26s 555ms/step - loss: 1.6168e-05 - acc: 1.0000 - val_loss: 1.4990 - val_acc: 0.8397
Epoch 4/15
47/47 [==============================] - 27s 572ms/step - loss: 0.0028 - acc: 0.9990 - val_loss: 1.6724 - val_acc: 0.8717
Epoch 5/15
47/47 [==============================] - 26s 561ms/step - loss: 0.0049 - acc: 0.9987 - val_loss: 1.2925 - val_acc: 0.8338
Epoch 6/15
47/47 [==============================] - 26s 561ms/step - loss: 0.0037 - acc: 0.9993 - val_loss: 1.4404 - val_acc: 0.8338
Epoch 7/15
47/47 [==============================] - 26s 548ms/step - loss: 1.6727e-04 - acc: 1.0000 - val_loss: 1.2920 - val_acc: 0.8426
Epoch 8/15
47/47 [==============================] - 26s 549ms/step - loss: 1.2401e-04 - acc: 1.0000 - val_loss: 1.3013 - val_acc: 0.8385
Epoch 9/15
47/47 [==============================] - 26s 561ms/st

47/47 [==============================] - 26s 553ms/step - loss: 0.0292 - acc: 0.9924 - val_loss: 1.5194 - val_acc: 0.8333
Epoch 9/15
47/47 [==============================] - 26s 547ms/step - loss: 0.0454 - acc: 0.9914 - val_loss: 2.4183 - val_acc: 0.7872
Epoch 10/15
47/47 [==============================] - 26s 554ms/step - loss: 0.0234 - acc: 0.9943 - val_loss: 2.1760 - val_acc: 0.8484
Epoch 11/15
47/47 [==============================] - 26s 547ms/step - loss: 0.0260 - acc: 0.9927 - val_loss: 3.6165 - val_acc: 0.7201
Epoch 12/15
12/47 [======>.......................] - ETA: 18s - loss: 0.0601 - acc: 0.9870

ResourceExhaustedError: OOM when allocating tensor with shape[64,1024,6,6]
	 [[Node: batch_normalization_95/FusedBatchNorm = FusedBatchNorm[T=DT_FLOAT, data_format="NHWC", epsilon=0.001, is_training=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense_1/Relu, batch_normalization_95/gamma/read, batch_normalization_95/beta/read, batch_normalization_86/Const_4, batch_normalization_86/Const_4)]]
	 [[Node: loss_13/mul/_23759 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_24875_loss_13/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'batch_normalization_95/FusedBatchNorm', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/